# Practical 4 - LSTM

In [1]:
import json
import pandas as pd
import re
import string
import tensorflow as tf
from tensorflow.keras import layers, losses
from keras.models import Model
from tensorflow.keras import optimizers
from keras.optimizers import Adam
from tensorflow.keras import layers, losses
import numpy as np
from keras import callbacks
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
with open("C:/Users/admin/Desktop/gen ai/full_format_recipes.json") as json_data:
    recipe_data = json.load(json_data)

In [3]:
filtered_data = ['Recipe for ' + x['title']+ ' | ' + ' '.join(x['directions'])
 for x in recipe_data
 if 'title' in x
 and x['title'] is not None
 and 'directions' in x
 and x['directions'] is not None
 ]
filtered_data

['Recipe for Lentil, Apple, and Turkey Wrap  | 1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool. 2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper. 3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.',
 'Recipe for Boudin Blanc Terrine with Red Onion Confit  | Combine first 9 ingredients in heavy medium 

In [5]:
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r' \1 ', s)
    s = re.sub(' +', ' ', s)
    return s

In [6]:
text_data = [pad_punctuation(x) for x in filtered_data]
text_data

['Recipe for Lentil , Apple , and Turkey Wrap | 1 . Place the stock , lentils , celery , carrot , thyme , and salt in a medium saucepan and bring to a boil . Reduce heat to low and simmer until the lentils are tender , about 30 minutes , depending on the lentils . ( If they begin to dry out , add water as needed . ) Remove and discard the thyme . Drain and transfer the mixture to a bowl ; let cool . 2 . Fold in the tomato , apple , lemon juice , and olive oil . Season with the pepper . 3 . To assemble a wrap , place 1 lavash sheet on a clean work surface . Spread some of the lentil mixture on the end nearest you , leaving a 1 - inch border . Top with several slices of turkey , then some of the lettuce . Roll up the lavash , slice crosswise , and serve . If using tortillas , spread the lentils in the center , top with the turkey and lettuce , and fold up the bottom , left side , and right side before rolling away from you . ',
 'Recipe for Boudin Blanc Terrine with Red Onion Confit | Co

In [7]:
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(32).shuffle(1000)

In [8]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(filtered_data)
total_words = len(tokenizer.word_index) + 1

In [9]:
vectorize_layer = layers.TextVectorization(
standardize = 'lower',
 max_tokens = 10000,
 output_mode = "int",
 output_sequence_length = 200 + 1,
 )

In [10]:
 vectorize_layer.adapt(text_ds)

In [11]:
vocab = vectorize_layer.get_vocabulary()
for index, word in enumerate(vocab):
    print(f"Index: {index}, Word: '{word}'")

Index: 0, Word: ''
Index: 1, Word: '[UNK]'
Index: 2, Word: '.'
Index: 3, Word: ','
Index: 4, Word: 'and'
Index: 5, Word: 'to'
Index: 6, Word: 'in'
Index: 7, Word: 'the'
Index: 8, Word: 'with'
Index: 9, Word: 'a'
Index: 10, Word: 'until'
Index: 11, Word: '1'
Index: 12, Word: 'minutes'
Index: 13, Word: '-'
Index: 14, Word: 'of'
Index: 15, Word: '2'
Index: 16, Word: 'for'
Index: 17, Word: 'heat'
Index: 18, Word: 'add'
Index: 19, Word: 'about'
Index: 20, Word: 'over'
Index: 21, Word: 'bowl'
Index: 22, Word: ';'
Index: 23, Word: '/'
Index: 24, Word: 'salt'
Index: 25, Word: 'into'
Index: 26, Word: 'recipe'
Index: 27, Word: '|'
Index: 28, Word: 'on'
Index: 29, Word: 'medium'
Index: 30, Word: 'large'
Index: 31, Word: 'mixture'
Index: 32, Word: '4'
Index: 33, Word: 'pepper'
Index: 34, Word: '('
Index: 35, Word: ')'
Index: 36, Word: '3'
Index: 37, Word: 'oil'
Index: 38, Word: 'is'
Index: 39, Word: 'water'
Index: 40, Word: 'transfer'
Index: 41, Word: 'or'
Index: 42, Word: 'stir'
Index: 43, Word: 

In [12]:
vectorized_text = vectorize_layer(text_data)
print(vectorized_text.shape)
print(vectorized_text.numpy())

(20111, 201)
[[  26   16 1733 ... 1072   96  165]
 [  26   16 6055 ...   23   15  106]
 [  26   16  335 ...    0    0    0]
 ...
 [  26   16  221 ...   13  250   78]
 [  26   16 1729 ...   90    2  120]
 [  26   16  771 ...    0    0    0]]


In [13]:
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y
train_ds = text_ds.map(prepare_inputs)


In [16]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(10000, 100)(inputs)
x = layers.LSTM(128, return_sequences=True)(x)
outputs = layers.Dense(10000, activation = 'softmax')(x)
lstm = Model(inputs, outputs)
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)
lstm.fit(train_ds, epochs=10, callbacks=[TextGenerator(index_to_word=vocab)])

Epoch 1/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 336s 532ms/step - loss: 5.0041
Epoch 2/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 333s 529ms/step - loss: 3.0668
Epoch 3/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 326s 517ms/step - loss: 2.5273
Epoch 4/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 327s 520ms/step - loss: 2.2604
Epoch 5/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 330s 524ms/step - loss: 2.1121
Epoch 6/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 326s 518ms/step - loss: 2.0015
Epoch 7/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 326s 518ms/step - loss: 1.9307
Epoch 8/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 348s 554ms/step - loss: 1.8739
Epoch 9/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 323s 514ms/step - loss: 1.8178
Epoch 10/10
629/629 ━━━━━━━━━━━━━━━━━━━━ 324s 515ms/step - loss: 1.7907
generated text


In [17]:
lstm= tf.keras.models.load_model('lstmforrecipe.h5')

In [18]:
import numpy as np
from tensorflow.keras import callbacks

class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {word: index for index, word in enumerate(index_to_word)}

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens=100, temperature=1.0):
        start_tokens = [self.word_to_index.get(x, 1) for x in start_prompt.split()]
        sample_token = None
        info = []

        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y = lstm.predict(x)  # Removed callback argument
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append({'prompt': start_prompt, 'word_probs': probs})

            if sample_token == 0:  # Avoid out-of-vocab issues
                break

            start_tokens.append(sample_token)
            start_prompt += " " + self.index_to_word.get(sample_token, "")

        print(f"\nGenerated Recipe:\n{start_prompt}\n")
        return start_prompt

In [19]:
# Assuming `tokenizer` was used in training
index_to_word = {index: word for word, index in tokenizer.word_index.items()}

# Initialize text generator
text_gen = TextGenerator(index_to_word)

# Generate a new recipe
generated_recipe = text_gen.generate("Recipe for Chocolate Cake", max_tokens=100, temperature=0.8)

print(generated_recipe)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━

In [ ]:
lstm prompt 

In [23]:
#lstm.save('lstmforrecipe.h5')

In [23]:
import numpy as np
from tensorflow.keras import callbacks

class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {word: index for index, word in enumerate(index_to_word)}

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs
    
    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [self.word_to_index.get(x, 1) for x in start_prompt.split()]
        sample_token = None
        info = []
        x = np.array([start_tokens])
        y = lstm.predict(x)

        for i in range(y.shape[1]):
            print('prompt',y[0][i])
            predicted_index = np.argmax(y[0][i])
            print(self.index_to_word[predicted_index])
        return info

TG = TextGenerator(index_to_word=vocab)

TG.generate("grilled chicken with boiled", max_tokens = 100, temperature = 1.0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
prompt [1.98614129e-04 1.04134866e-04 2.47228105e-04 ... 7.18244337e-05
 6.60712394e-05 1.12292284e-04]
red
prompt [1.80444855e-04 1.57217903e-04 4.47857863e-04 ... 5.89734300e-05
 5.48470416e-05 1.08756205e-04]
breast
prompt [4.3405304e-04 8.9240615e-04 4.6495826e-04 ... 5.4992784e-06 2.5486710e-05
 1.9348039e-05]
garlic
prompt [8.8889159e-05 6.1222696e-04 1.8570870e-04 ... 1.8872883e-07 5.9367227e-07
 4.3752129e-06]
eggs


[]

In [24]:
class TextGenerator():
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
        word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs


    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [self.word_to_index.get(x, 1) for x in start_prompt.split()]
        sample_token = None
        info = []
        x = np.array([start_tokens])
        y = lstm.predict(x)
        sample_token, probs = self.sample_from(y[0][-1], temperature)
        info.append({'prompt': start_prompt , 'word_probs': probs})
        start_tokens.append(sample_token)
        start_prompt = start_prompt + ' ' + self.index_to_word[sample_token]
        print(start_prompt)

TG = TextGenerator(index_to_word=vocab)

TG.generate("grilled chicken with boiled", max_tokens = 100, temperature = 0.1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
grilled chicken with boiled eggs
